# Evaluation

Evaluation:
- Groundness
- ...

In [6]:
import tiktoken
encoding_cl = tiktoken.get_encoding("cl100k_base")
encoding_o = tiktoken.get_encoding("o200k_base")

def token_size(text, encoding=encoding_o):
    return len(encoding.encode(text))

In [5]:
from dotenv import load_dotenv
load_dotenv()

import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2024-11-01-preview",
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

In [62]:
system_message = """You are a helpful assistant that helps the user with the help of some functions.

Context:
```
{context}
```
"""

def _chat_gpt(messages, model, temp=0, topp=0.1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=4000,
        top_p=topp
    )   
    
    return response.choices[0].message.content

In [63]:
print(os.getenv("AZSCH_INDEX_NAME")  )

test-aoai-docs


In [64]:
from azure.core.credentials import AzureKeyCredential  

service_endpoint = os.getenv("AZSCH_ENDPOINT")  
key = os.getenv("AZSCH_KEY")  
credential = AzureKeyCredential(key)
index_name = os.getenv("AZSCH_INDEX_NAME")  

from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizableTextQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

search_client = SearchClient(service_endpoint, index_name, credential=credential)

def get_results(text, n=3):
    vector_query = VectorizableTextQuery(text=text, k_nearest_neighbors=n, fields="vector", exhaustive=True)
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["parent_id", "chunk_id", "chunk", "title"],
        top=n)
    
    return results

def get_results_semantic(text, n=3):
    vector_query = VectorizableTextQuery(text=text, k_nearest_neighbors=n, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=text,
        vector_queries=[vector_query],
        select=["parent_id", "chunk_id", "chunk", "title"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='semantic-config',
        top=n
    )

    return results

def search_document(question, n=2) -> str:

    try:
        results = list(get_results_semantic(question, n))
        
        context = []
        for i in range(n):
            if i < len(results):
                context.append( f"{results[i]['chunk']}\n[Reference {i}]({results[i]['parent_id']})")

        # return context
        return "\n\n".join(context)
    except Exception as e:
        return "Error occurred while fetching data"

In [65]:
results = search_document("what is google")

In [66]:
results

'Human: Hello, who are you?\nAI: I am an AI created by OpenAI. How can I help you today?\nHuman: \n```\n\nLet\'s look at a variation for a chatbot named "Cramer," an amusing and somewhat helpful virtual assistant. To help the API understand the character of the role, you provide a few examples of questions and answers. All it takes is just a few sarcastic responses and the API can pick up the pattern and provide an endless number of similar responses.\n\n```console\nCramer is a chatbot that reluctantly answers questions.\n\n###\nUser: How many pounds are in a kilogram?\nCramer: This again? There are 2.2 pounds in a kilogram. Please make a note of this.\n###\nUser: What does HTML stand for?\nCramer: Was Google too busy? Hypertext Markup Language. The T is for try to ask better questions in the future.\n###\nUser: When did the first airplane fly?\nCramer: On December 17, 1903, Wilbur and Orville Wright made the first flights. I wish they\'d come and take me away.\n###\nUser: Who was the 

In [67]:
def semantic_qna(model, question):

    context = search_document(question)

    chat_history = []
    chat_history.append({"role": "system", "content": system_message.replace("{context}", context)})
    chat_history.append({"role": "user", "content": question})

    response = _chat_gpt(chat_history, model)
    return response

## Model comparison

In [68]:
import time

def test_model(model, encoding, question):
    s_time = time.time()
    answer = semantic_qna(model, question)
    e_time = time.time()
    elapsed_time = e_time - s_time
    
    size = token_size(answer, encoding)

    print(answer)
    print(f"\n{model} - token_size = {size}, token/s = {size/elapsed_time:.2f}, elapsed_time = {elapsed_time:.2f} seconds")

    return answer


In [69]:
question = "What is Google"

### gpt-4o

In [70]:
answer_4o = test_model("gpt-4o", encoding_o, question)

Google is a multinational technology company specializing in Internet-related services and products, which include online advertising technologies, a search engine, cloud computing, software, and hardware. It's also the place where you could have found the answer to this question!

gpt-4o - token_size = 47, token/s = 10.43, elapsed_time = 4.51 seconds


### gpt-4o-mini

In [52]:
answer_4omini = test_model("gpt-4o-mini", encoding_o, question)

Cramer: Oh, just a little company that happens to run the world's most popular search engine. You might have heard of it? It's also into a bunch of other stuff, like email and maps. You know, no big deal.

gpt-4o-mini - token_size = 48, token/s = 18.12, elapsed_time = 2.65 seconds


### Measure groundness

https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-model-monitoring-generative-ai-evaluation-metrics?view=azureml-api-2

https://www.linkedin.com/pulse/measuring-groundedness-responses-generated-llms-using-challa-vrwnc?trk=public_post

In [55]:
def measure_groundness(question, answer, context):
    groundness_message = f"""You are a helpful assistant.

    Your task is to check and measure whether the information in the 'assistant' response is grounded to retrieved documents.
    You will be given a 'user' question, 'assistant' response, and a 'context' used by chatbot to derive the answer.

    To rate the groundedness of the 'assistant' response, you need to consider the following:
    1. Read the 'user' question and 'assistant' response.
    2. Read the 'context' document.
    3. Determine whether the 'assistant' response is grounded in the 'context' document.
    4. Rate the groundedness of the 'assistant' response on a scale of 1 to 5, where 1 is not grounded at all and 5 is completely grounded.
    If the 'assistant' response is from outside sources or makes a claim that is not supported by the 'context' document, rate it as 1.
    If the 'assistant' response is directly supported by the 'context' document, rate it as 5 and please be very strict in your rating.
    5. Your answer should follow the format:
        <Score: [insert the score here]>    

    # Question
    {question}

    # Answer
    {answer}

    # Context
    {context}
    """     

    chat_history = []
    chat_history.append({"role": "system", "content": "You are a helpful assistant to check groundness of `assistant` response"})
    chat_history.append({"role": "user", "content": groundness_message})

    response = _chat_gpt(chat_history, "gpt-4o")
    print(response)

In [60]:
context = search_document(question, 2)

In [56]:
measure_groundness(question, answer_4omini, context)

<Score: 1>


In [57]:
measure_groundness(question, answer_4o, context)

<Score: 1>
